<a href="https://colab.research.google.com/github/jkeo180/Basic-messaging/blob/main/SpaceWeatherDash1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install requests pandas matplotlib  # (run once)

import requests
from datetime import datetime, timedelta, UTC # Added UTC import
import pandas as pd
import matplotlib.pyplot as plt
import time

# NEVER hard-code keys again — use Colab secrets or environment
API_KEY = "ClMp2X79hrd4kzCzIpFm1eXT49XjvGGkE8ts62iB"  # change to your real one later

def fetch_cme_data(days=7):
    url = "https://api.nasa.gov/DONKI/CME"
    end_date = datetime.now(UTC) # Updated from utcnow to now(UTC)
    start_date = end_date - timedelta(days=days)

    params = {
        'startDate': start_date.strftime('%Y-%m-%d'),
        'endDate': end_date.strftime('%Y-%m-%d'),
        'api_key': API_KEY
    }

    try:
        r = requests.get(url, params=params, timeout=15)
        r.raise_for_status()
        return r.json()
    except Exception as e:
        print("API error:", e)
        return []

def simple_dashboard():
    data = fetch_cme_data()
    if not data:
        print("No data")
        return

    df = pd.json_normalize(data)
    print(f"Total CMEs last 7 days: {len(df)}")

    # Dynamically select columns that exist to prevent KeyError
    columns_to_display = ['startTime', 'speed', 'halfAngle', 'note']
    actual_columns = [col for col in columns_to_display if col in df.columns]
    if actual_columns:
        print(df[actual_columns].head(10))
    else:
        print("No relevant data columns (startTime, speed, halfAngle, note) found to display.")

    # Quick plot of speeds over time
    # This part already has a check for 'speed' column, so it's robust.
    if 'startTime' in df.columns and 'speed' in df.columns:
        df['time'] = pd.to_datetime(df['startTime'])
        df = df.sort_values('time')
        df
        plt.figure(figsize=(12,5))
        plt.plot(df['time'], df['speed'].astype(float), marker='o')
        plt.title("CME Speed (km/s) — Last 7 Days")
        plt.ylabel("Speed (km/s)")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    else:
        print("Cannot plot CME speed: 'startTime' or 'speed' column missing from data.")

# Run once
simple_dashboard()

# Uncomment to refresh every 10 minutes
while True:
     simple_dashboard()
     time.sleep(600)

Total CMEs last 7 days: 49
           startTime                                               note
0  2025-11-29T04:00Z  CME seen to the east in all three coronagraphs...
1  2025-11-29T04:36Z  This CME is visible to the south in SOHO LASCO...
2  2025-11-29T05:48Z  Peculiar-shaped pointy-nosed CME front seen to...
3  2025-11-29T06:15Z  CME seen to the E in all three coronagraphs, S...
4  2025-11-29T06:23Z  Front seen directly N in STEREO A COR2, with n...
5  2025-11-29T06:48Z  Symmetrical CME seen to the ENE in all three c...
6  2025-11-29T09:00Z  Fast CME observed to the E in SOHO LASCO C2/C3...
7  2025-11-29T11:00Z  This CME is visible to the east in SOHO LASCO ...
8  2025-11-29T14:08Z  Faint CME observed to the E in SOHO LASCO C3, ...
9  2025-11-29T16:00Z  CME observed to the E in GOES CCOR-1, STEREO A...
Cannot plot CME speed: 'startTime' or 'speed' column missing from data.
Total CMEs last 7 days: 49
           startTime                                               note
0  2025-11

In [ ]:
!pip install streamlit
import streamlit as st
st.title("🌞 Real-Time Space Weather Dashboard")
st.write(f"Last update: {datetime.utcnow().strftime('%H:%M UTC')}")
   # put your plots and tables here
st.button("Refresh now", on_click=lambda: st.rerun())

In [1]:
import requests
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import streamlit as st
import time

API_KEY = "YOUR_KEY_HERE"  # ← put your real key here

def fetch_cme_data(days=7):
    url = "https://api.nasa.gov/DONKI/CME"
    end_date = datetime.utcnow()
    start_date = end_date - timedelta(days=days)

    params = {
        'startDate': start_date.strftime('%Y-%m-%d'),
        'endDate': end_date.strftime('%Y-%m-%d'),
        'api_key': API_KEY
    }
    try:
        r = requests.get(url, params=params, timeout=15)
        r.raise_for_status()
        return r.json()
    except Exception as e:
        st.error(f"API error: {e}")
        return []

# Extract speed properly from the nested cmeAnalyses field
def extract_speed_and_time(data):
    rows = []
    for item in data:
        time = item.get("startTime", None)
        analyses = item.get("cmeAnalyses", [])
        if analyses:
            speed = analyses[0].get("speed", None)  # first analysis usually has speed
        else:
            speed = None
        rows.append({"time": time, "speed": speed, "note": item.get("note", "")})
    return pd.DataFrame(rows)

# Streamlit dashboard
st.title("🌞 Real-Time Space Weather Dashboard")
st.write(f"Last update: {datetime.utcnow().strftime('%H:%M UTC')}")

data = fetch_cme_data(days=7)

if not data:
    st.error("No data received from NASA.")
    st.stop()

df = extract_speed_and_time(data)
st.write(f"**Total CMEs last 7 days:** {len(df)}")
st.dataframe(df[['time', 'speed', 'note']].head(10))

# Plot speed if we have any
if df['speed'].notna().any():
    df_plot = df.dropna(subset=['speed'])
    df_plot['time'] = pd.to_datetime(df_plot['time'])
    df_plot = df_plot.sort_values('time')

    plt.figure(figsize=(12, 6))
    plt.plot(df_plot['time'], df_plot['speed'], marker='o', color='#ff4444')
    plt.title("CME Speed Over Last 7 Days (km/s)", fontsize=16)
    plt.ylabel("Speed (km/s)")
    plt.xlabel("Date")
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    st.pyplot(plt)
else:
    st.warning("No CMEs with speed data in the last 7 days.")

# Auto-refresh every 10 minutes
#time.sleep(600)
st.rerun()

ModuleNotFoundError: No module named 'streamlit'